Maternal Health Data And Health Risk Level

Question: To what extent can we use model data from “sensing enabled technology”: blood pressure, blood glucose level, body temperature and heart rate, to classify and predict Bangladeshi women’s maternal health risk level?

Fetal and maternal fatality is a pervasive problem especially in developing nations, and especially felt by disadvantaged classes. The Maternal Health Risk Dataset from UC Irvine Machine Learning Repository has tracked sensory data from pregnant women in rural villages in Bangladesh regarding health parameters like blood pressure and also their respective maternal health risk. 

Our data has the following variables (columns):

- Age 
- Systolic Blood Pressure (SystolicBP)
- Diastolic Blood Pressure (DiastolicBP)
- Blood Glucose Level (BS)
- Body Temperature (BodyTemp)
- Resting Heart Rate (HeartRate)
- Predicted Risk Intensity Level during pregnancy (RiskLevel)

We will use tidymodels to perform K-nearest neighbours to classify maternal health risk as either high, medium, or low, as predicted by all the remaining health variables. We will be doing cross-validation by splitting the same data into training and test sets to get a more robust estimate of accuracy. Understanding the association between health data and maternal risk can help medical professionals advise women on pregnancy and also help them understand what risk factors to look out for to prepare for them in advance.  

References:

10.24432/C5DP5D

Ahmed, M., Kashem, M.A., Rahman, M., & Khatun, S. (2020). Review and Analysis of Risk Factor of Maternal Health in Remote Area Using the Internet of Things (IoT).


In [2]:
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [3]:
mat_health_risk <- read_csv("https://raw.githubusercontent.com/aaronardenma/dsci-100-2023w1-group-41/main/Maternal_Health_Risk_Data_Set.csv")

mat_health_risk

Rows: 1014 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): RiskLevel
dbl (6): Age, SystolicBP, DiastolicBP, BS, BodyTemp, HeartRate

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
25,130,80,15.00,98,86,high risk
35,140,90,13.00,98,70,high risk
29,90,70,8.00,100,80,high risk
30,140,85,7.00,98,70,high risk
35,120,60,6.10,98,76,low risk
23,140,80,7.01,98,70,high risk
23,130,70,7.01,98,78,mid risk
35,85,60,11.00,102,86,high risk
32,120,90,6.90,98,70,mid risk


In [7]:
set.seed(1) 

mat_health_risk_split <- initial_split(mat_health_risk, prop = 0.75, strata = RiskLevel)  
data_train <- training(mat_health_risk_split)   
data_test <- testing(mat_health_risk_split)

data_train

Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
25,130,80,15.00,98,86,high risk
35,140,90,13.00,98,70,high risk
23,140,80,7.01,98,70,high risk
25,140,100,7.01,98,80,high risk
40,140,100,18.00,98,90,high risk
25,140,100,6.80,98,80,high risk
23,140,90,6.80,98,70,high risk
34,85,60,11.00,102,86,high risk
50,140,90,15.00,98,90,high risk


In [8]:
unique_cases <- data_train |>
select(RiskLevel) |>
unique()

n_observations_1 <- data_train |>
filter(RiskLevel == "high risk") |>
nrow()

n_observations_2 <- data_train |>
filter(RiskLevel == "mid risk") |>
nrow()

n_observations_3 <- data_train |>
filter(RiskLevel == "low risk") |>
nrow()

classes <- c(slice(unique_cases, 1), slice(unique_cases, 2), slice(unique_cases, 3))

observations <- c(n_observations_1, n_observations_2, n_observations_3)
case_ob <- tibble(classes = classes, n_observations = observations)
case_ob

classes,n_observations
<named list>,<int>
high risk,204
low risk,252
mid risk,304


In [9]:
predictor_means <- data_train |>
select(-RiskLevel) |>
map_df(mean, na.rm=TRUE)

predictor_means

Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29.36053,113.0947,76.44605,8.773211,98.65789,74.27368


In [10]:
na <- data_train |>
is.na() |>
unique()

na #no missing values in dataset

Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
